# Recurrent Neural Networks

Grzegorz Statkiewicz, Mateusz Matukiewicz

# Overview

The structure of the direcotry should be as follows:

```
.
├── data
│   ├── train.pkl
│   └── test_no_target.pkl
└── main.ipynb
```



## Setup

Select the device to use

In [1]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

Using device: mps


## Data preparation

Load the data

In [3]:
train_path = "data/train.pkl"

In [ ]:
import pickle

with open(train_path, "rb") as f:
    train_data = pickle.load(f)

print(f"Loaded {len(train_data)} training samples.")

Loaded 2939 training samples.
(array([-1., -1., -1., ..., -1., -1., -1.], shape=(3489,)), 1)


Print sample data

In [10]:
import random


idx = random.randint(0, len(train_data) - 1)
print(f"Sample data: {train_data[idx]}")

Sample data: (array([  0., 114.,  12.,  12.,  28.,  28.,  28.,  78.,  13.,  28.,  13.,
        28.,  28.,  12.,  92.,  12.,  12.,  28.,  12.,  13.,  13., 125.,
        28.,  28.,  28.,  78.,  13.,  28.,  13.,  93.,  28.,  28.,  28.,
        28.,  13., 124.,  12.,  73.,  12.,  12.,  28.,  28.,  28.,  78.,
        13.,  28.,  13.,  28.,  28.,  12.,  92.,  12.,  12.,  28.,  12.,
        13.,  13., 125.,  28.,  28.,  28.,  78.,  13.,  28.,  13.,  93.,
        28.,  28.,  28.,  28.,  13., 124.,  12.,  73.,  12.,  12.,  28.,
        28.,  28.,  78.,  13.,  28.,  13.,  79.,  79.,  93.,  13.,  13.,
        92.,  92.,  15.,  15., 124., 124.,  15.,  79.,  79., 159., 125.,
        28.,  79.,  15.,  15.,  15., 159.,  28.,  28.,  28.,  28.,  13.,
        13.,  28.,  73.,  73.,  12.,  12.,  28.,  28.,  28.,  78.,  13.,
        28.,  13.,  28.,  28.,  12.,  92.,   0.]), 0)
